In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmnist-alphabet-94-characters/94_character_TMNIST.csv


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, ReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

2024-04-24 05:00:38.956429: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 05:00:38.956616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 05:00:39.163045: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Hyperparameters
batch_size = 128 # How many independent images should we process in parallel.
n_hidden = 128 # Define the number of hidden layers in dense neural net layer.
max_iters = 10000 # Number of training iterations of the model.

In [4]:
# load data
df = pd.read_csv("/kaggle/input/tmnist-alphabet-94-characters/94_character_TMNIST.csv")
df.head()

,names,labels,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,784
0,Salsa-Regular,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MouseMemoirs-Regular,D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Creepster-Regular,f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SeoulNamsan-Light,/,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HachiMaruPop-Regular,F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# input(X) and output labels(y)

# Dividing by 255 to normalize the data bring to the range(0, 1)
X=df.drop(['names','labels'],axis=1).values.reshape(df.shape[0], 1, 28, 28) / 255
y=df['labels']

In [6]:
chars = sorted(list(df['labels'].unique()))
vocab_size = len(chars) # Number of unique characters in the input.

itos = {i:ch for i,ch in enumerate(chars)}
stoi = {ch:i for i,ch in enumerate(chars)}

# Map the labels for string to integer.
y_upd = np.array([stoi[ch] for ch in y])

print(vocab_size)

94


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_upd, test_size = 0.2, random_state = 1337)

In [8]:
# Convert the data to torch tensors
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)

# Convert the label data to one hot encoding vectors.
y_train_one_hot = F.one_hot(torch.from_numpy(y_train), num_classes=vocab_size)
y_test_one_hot = F.one_hot(torch.from_numpy(y_test), num_classes=vocab_size)

In [9]:
# Get batch_size rows of train/test data for their respective purposes.
def get_batch(split):
    X_c = X_train if split == "train" else X_test
    y_c = y_train_one_hot if split == "train" else y_test_one_hot
    ix = torch.randint(len(X_c), (batch_size, ))
    x_batch = torch.tensor(torch.stack([X_c[i] for i in ix], dim=0), dtype=torch.float32)
    y_batch = torch.tensor(torch.stack([y_c[i] for i in ix], dim=0), dtype=torch.float32)
    return x_batch.to(device), y_batch.to(device)

In [10]:
# Estimate the loss of the model on a small slice of test set(batch_size(128) rows).
@torch.no_grad()
def estimate_loss():
    correct = 0
    model.eval()
    x_t, y_t = get_batch("test")
    for inp, label in zip(x_t, y_t):
        inp = inp.view(1, -1, inp.shape[1], inp.shape[2])
        out = model(inp)
        predicted = torch.argmax(out, dim=1)
        label_predicted = torch.argmax(label)
        correct += (predicted==label_predicted).sum().item()
    total = y_t.shape[0]
    accuracy = correct * 100.0 / total
    model.train()
    return accuracy

In [11]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class ConvNet(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3), # (B, 1, 28, 28) --> (B, 32, 26, 26)
            nn.ReLU(), # (B, 32, 26, 26) --> (B, 32, 26, 26)
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3), # (B, 32, 26, 26) --> (B, 64, 24, 24)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # (B, 64, 24, 24) --> (B, 64, 12, 12)
            nn.Flatten(start_dim=1, end_dim=-1), # (B, 64, 12, 12) --> (B, 64 * 12 * 12)
            nn.Linear(64 * 12 * 12, n_hidden), # (B, 9216) --> (B, n_hidden)
            nn.BatchNorm1d(n_hidden), # Applied batch norm to make the model more resilient.
            nn.ReLU(),
            nn.Linear(n_hidden, vocab_size), # (B, n_hidden) --> (B, vocab_size)
        )
        
    
    def forward(self, x):
        return self.net(x)
    
model = ConvNet()
model = model.to(device)


In [12]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-4)

for i in range(max_iters):
    xb, yb = get_batch("train")
    logits = model(xb)
    loss = F.cross_entropy(logits, yb)
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if (i % 500 == 0):
        accuracy = estimate_loss()
        print(f"step: {i}, train loss: {loss:.4f}, test accuracy: {accuracy:.4f}%")

/tmp/ipykernel_19/434405513.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_batch = torch.tensor(torch.stack([X_c[i] for i in ix], dim=0), dtype=torch.float32)
/tmp/ipykernel_19/434405513.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_batch = torch.tensor(torch.stack([y_c[i] for i in ix], dim=0), dtype=torch.float32)


step: 0, train loss: 4.6321, test accuracy: 1.5625%
step: 500, train loss: 0.5139, test accuracy: 96.0938%
step: 1000, train loss: 0.2690, test accuracy: 90.6250%
step: 1500, train loss: 0.2987, test accuracy: 95.3125%
step: 2000, train loss: 0.1965, test accuracy: 92.9688%
step: 2500, train loss: 0.1960, test accuracy: 88.2812%
step: 3000, train loss: 0.1931, test accuracy: 91.4062%
step: 3500, train loss: 0.1694, test accuracy: 90.6250%
step: 4000, train loss: 0.1188, test accuracy: 92.1875%
step: 4500, train loss: 0.1274, test accuracy: 93.7500%
step: 5000, train loss: 0.1314, test accuracy: 91.4062%
step: 5500, train loss: 0.1383, test accuracy: 96.8750%
step: 6000, train loss: 0.1761, test accuracy: 90.6250%
step: 6500, train loss: 0.0947, test accuracy: 89.8438%
step: 7000, train loss: 0.1443, test accuracy: 93.7500%
step: 7500, train loss: 0.0544, test accuracy: 92.9688%
step: 8000, train loss: 0.0983, test accuracy: 95.3125%
step: 8500, train loss: 0.0828, test accuracy: 92.187

In [13]:
# Loss on the entire test set. 
# Accuracy comes to 96.82%

with torch.no_grad():
    correct = 0
    model.eval()
    x_t, y_t = torch.tensor(X_train, dtype=torch.float32).to(device), y_train_one_hot.to(device)
    for inp, label in zip(x_t, y_t):
        inp = inp.view(1, -1, inp.shape[1], inp.shape[2])
        out = model(inp)
        predicted = torch.argmax(out, dim=1)
        label_predicted = torch.argmax(label)
        correct += (predicted==label_predicted).sum().item()
    total = y_t.shape[0]
    accuracy = correct * 100.0 / total
    model.train()
    print(f"Accuracy on the entire test set: {accuracy:.2f}%")

/tmp/ipykernel_19/2336300992.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_t, y_t = torch.tensor(X_train, dtype=torch.float32).to(device), y_train_one_hot.to(device)


Accuracy on the entire test set: 97.00%
